In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 载入数据集
mnist = input_data.read_data_sets(r"C:\Users\zdwxx\Downloads\Compressed\MNIST_data", one_hot=True)

def varible_summaries(var, name):
    
    with tf.name_scope(name):
        
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean) # 平均值
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev) # 标准差
        tf.summary.scalar("max", tf.reduce_max(var)) #最大值
        tf.summary.scalar("min", tf.reduce_min(var)) # 最小值
        tf.summary.histogram("histogram", var) # 直方图

# 定义每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 初始化权值函数
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) # 生成一个截断的正态分布
    return tf.Variable(initial)

# 初始化偏置值函数
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x, W):
#   x input tensor of shape:`[filter_height * filter_wid3th * in_channels, output_channels]`.
# `[filter_height, filter_width, in_channels, out_channels]`
# `strides[0] = strides[3] = 1,strides[1]->x方向上的补偿，[2]->y方向上的补偿
# padding: A string from:"SAME", "VAILD"
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

# 池化层
def max_pool_2x2(x):
# ksize[1,x,y,1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# 改变x的格式转为4D的向量[batch, in_height, in_widch, in_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1]) # [批次，高，宽，通道]

# 初始化第一个卷积层的权值和偏置值
W_conv1 = weight_variable([5, 5, 1, 32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征
varible_summaries(W_conv1, "W_conv1")
b_conv1 = bias_variable([32]) # 每一个卷积核的偏置值
varible_summaries(b_conv1, "b_conv1")


# 把x_image的权值进行卷积，加上偏置值，然后应用于relu激活函数

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 进行max_pool
varible_summaries(h_pool1, "h_pool1")

# 初始化第二个卷积层的权值和偏置值
W_conv2 = weight_variable([5, 5, 32, 64]) # 5*5的采样窗口，64个卷积核从32个平面抽取特征
varible_summaries(W_conv2, "W_conv2")
b_conv2 = bias_variable([64]) # 每一个卷积核的偏置值
varible_summaries(b_conv2, "b_conv2")

# 把x_image的权值进行卷积，加上偏置值，然后应用于relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # 进行max_pool
varible_summaries(h_pool2, "h_pool2")

# 28*28的图片第一次卷积后还是28*28，第一次池化后变成了14*14
# 第二次卷积后为14*14，第二次池化后变成了7*7
# 通过上面的操作后得到了64张7*7的平面

# 初始化第一个全连接层的权值
W_fc1 = weight_variable([7*7*64, 1024])
varible_summaries(W_fc1, "W_fc1")
b_fc1 = bias_variable([1024])
varible_summaries(b_fc1, "b_fc1")

# 把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# 求出第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 使用dropout防止过拟合
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
varible_summaries(h_fc1_drop, "h_fc1_dropout")

# 初始化第二个全连接层
W_fc2 = weight_variable([1024, 10])
varible_summaries(W_fc2, "W_fc2")
b_fc2 = bias_variable([10])
varible_summaries(b_fc2, "b_fc2")

# 计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
tf.summary.scalar("loss", loss)
# loss = tf.reduce_mean(tf.square(y - prediction))

# Adam优化器
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# 结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax返回1维张量中最大的值所在的位置
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))#cast转换类型，True->1.0, False->0.0
tf.summary.scalar("accuacy", accuracy)

# 合并所有的summary
merged = tf.summary.merge_all()

# 保存模型
saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer()) # first thing first初始化变量
    writer = tf.summary.FileWriter(r"C:\logs", sess.graph)

    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size) #类似于read，一次读取100张图片
            sess.run(train_step, feed_dict={x : batch_xs, y : batch_ys, keep_prob : 0.7})
            
        summary = sess.run(merged, feed_dict={x : batch_xs, y : batch_ys, keep_prob : 1.})
        writer.add_summary(summary, epoch)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob : 1.})
        print("第", epoch, "个周期", "准确率是", acc)
    
    saver.save(sess, "c:/model/cnn_mnist/my_mnist_net.ckpt")

Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-labels-idx1-ubyte.gz
第 0 个周期 准确率是 0.8663
第 1 个周期 准确率是 0.9709
第 2 个周期 准确率是 0.9796
第 3 个周期 准确率是 0.9795
第 4 个周期 准确率是 0.985
第 5 个周期 准确率是 0.9856
第 6 个周期 准确率是 0.9884
第 7 个周期 准确率是 0.9888
第 8 个周期 准确率是 0.9895
第 9 个周期 准确率是 0.9895
第 10 个周期 准确率是 0.99
第 11 个周期 准确率是 0.9896
第 12 个周期 准确率是 0.991
第 13 个周期 准确率是 0.9908
第 14 个周期 准确率是 0.9906
第 15 个周期 准确率是 0.9917
第 16 个周期 准确率是 0.991
第 17 个周期 准确率是 0.9911
第 18 个周期 准确率是 0.9903
第 19 个周期 准确率是 0.9909
第 20 个周期 准确率是 0.9918
